### **Import the dataset**

In [1]:
!pip3 install scikit-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.13 -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("datasets/Bank_Market_Dataset.csv")
df.head(5)

,Unnamed: 0,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
df = df.drop(columns= "Unnamed: 0")
df.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [6]:
num_cols = df.select_dtypes(include=["int64"]).columns
object_cols = df.select_dtypes(include=["object"]).columns

In [7]:
num_cols

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object')

In [8]:
# remove the y from object_cols
object_cols = object_cols[:-1]

In [9]:
object_cols

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')

In [10]:
#convert the target variable to binary
df["y"] = df["y"].apply(lambda x: 1 if x == "yes" else 0)

In [11]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous,y
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323,0.116985
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441,0.321406
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000,1.000000


### **Preprocess the dataset**

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),
    ('std_scaler', StandardScaler())
])

object_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('onehot', OneHotEncoder())
])

In [13]:
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('obj', object_pipeline, object_cols)
])

In [14]:
X = df.drop(columns="y")
y = df["y"]

In [15]:
#applying the preprocessor to X.T
X = preprocessor.fit_transform(X)

In [16]:
X.shape

(45211, 51)

### **Ready the model inputs**

In [17]:
#uncomment this to install PyTorch for Apple Silicon

# !pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

In [18]:
#convert the data to tensors
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.int64)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

In [20]:
#just to verify the shape and data type :)

print(X_train.shape)
print(X_train.dtype)

torch.Size([36168, 51])
torch.float32


In [21]:
#create TensorDatasets

trainDataset = TensorDataset(X_train, y_train)
testDataset = TensorDataset(X_test, y_test)

#creating the DataLoaders

trainLoader = DataLoader(trainDataset, batch_size= 2, shuffle=True)
testLoader = DataLoader(testDataset, batch_size=2, shuffle=True)

### **Define the Feed-Forward Neural Network**

In [22]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedForwardNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
    def forward(self, x):
        return self.network(x)

In [23]:
#configure the model

input_size = X_train.shape[1]
hidden_size = 32
output_size = 2

model = FeedForwardNetwork(input_size, hidden_size, output_size)

In [24]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [25]:
# Train the model
epochs = 25
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for X_batch, y_batch in trainLoader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
    
    epoch_loss = running_loss / len(trainLoader)
    epoch_accuracy = correct / total
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in testLoader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch 1, Loss: 0.2256, Accuracy: 0.9009
Epoch 2, Loss: 0.2117, Accuracy: 0.9062
Epoch 3, Loss: 0.2055, Accuracy: 0.9079
Epoch 4, Loss: 0.2017, Accuracy: 0.9086
Epoch 5, Loss: 0.1988, Accuracy: 0.9106
Epoch 6, Loss: 0.1959, Accuracy: 0.9112
Epoch 7, Loss: 0.1951, Accuracy: 0.9121
Epoch 8, Loss: 0.1930, Accuracy: 0.9130
Epoch 9, Loss: 0.1916, Accuracy: 0.9127
Epoch 10, Loss: 0.1907, Accuracy: 0.9139
Epoch 11, Loss: 0.1899, Accuracy: 0.9150
Epoch 12, Loss: 0.1894, Accuracy: 0.9140
Epoch 13, Loss: 0.1887, Accuracy: 0.9154
Epoch 14, Loss: 0.1887, Accuracy: 0.9148
Epoch 15, Loss: 0.1870, Accuracy: 0.9158
Epoch 16, Loss: 0.1868, Accuracy: 0.9160
Epoch 17, Loss: 0.1864, Accuracy: 0.9158
Epoch 18, Loss: 0.1867, Accuracy: 0.9163
Epoch 19, Loss: 0.1860, Accuracy: 0.9158
Epoch 20, Loss: 0.1851, Accuracy: 0.9164
Epoch 21, Loss: 0.1849, Accuracy: 0.9163
Epoch 22, Loss: 0.1846, Accuracy: 0.9177
Epoch 23, Loss: 0.1845, Accuracy: 0.9165
Epoch 24, Loss: 0.1847, Accuracy: 0.9172
Epoch 25, Loss: 0.1839, A